# **Introduction**

This notebook serves as a method of testing the creation of parameterized MuJoCo environments programatically.

# **Imports**

This section imports the necessary packages.

In [145]:
# imports:
import mujoco as mj
import mujoco.viewer
import numpy as np

# **Define Parameters**

This section defines the parameters for use in the creation of the MuJoCo environment.

In [146]:
### MODEL SETTINGS ###
MODEL_NAME = "my_envrionment"

### COMPILER SETTINGS ###
COMPILER_ANGLE = 1

### OPTION SETTINGS ###
TIMESTEP = 0.001
INTEGRATOR = 2  # this is RK4
GRAVITY = np.array([0, 0, -9.81], dtype = np.float32)

### DEFAULT SETTINGS ###
JOINT_DAMPING = 0.25

### VISUAL SETTINGS ###
ZNEAR = 0.01
ZFAR = 50
SHADOWSIZE = 4096

### ASSET SETTINGS ###
# skybox:
SKYBOX_NAME = "sky"
SKYBOX_TYPE = mj.mjtTexture.mjTEXTURE_SKYBOX
SKYBOX_BUILTIN = mj.mjtBuiltin.mjBUILTIN_GRADIENT
SKYBOX_RGB1 = np.array([1.0, 1.0, 1.0], dtype = np.float32)
SKYBOX_RGB2 = np.array([0.6, 0.8, 1.0], dtype = np.float32)
SKYBOX_WIDTH = 256
SKYBOX_HEIGHT = 256

### WORLDBODY SETTINGS ###
# light:
LIGHT_NAME = "overhead_light"
LIGHT_POS = [0, 0, 5]
LIGHT_DIFFUSE = [0.9, 0.9, 0.9]
LIGHT_SPECULAR = [0, 0, 0]
LIGHT_AMBIENT = [0, 0, 0]

# camera:
CAMERA_NAME = "overhead_camera"
CAMERA_POS = [0, 0, 3]

# walls:
WALL_TYPE = mj.mjtGeom.mjGEOM_BOX
WALL_CONTYPE = 1
WALL_CONAFFINITY = 1
WALL_THICKNESS = 0.025
WALL_HEIGHT = 0.05
WALL_INDICES = [("right", [1, 0, 0]),
                ("left", [-1, 0, 0]),
                ("front", [0, 1, 0]),
                ("back", [0, -1, 0])]

# ground-plane:
GROUND_NAME = "ground_plane"
GROUND_TYPE = mj.mjtGeom.mjGEOM_PLANE
GROUND_CONTYPE = 1
GROUND_CONAFFINITY = 1
GROUND_INTERNAL_LENGTH = 1
GROUND_ACTUAL_LENGTH = GROUND_INTERNAL_LENGTH + 2 * WALL_THICKNESS
GROUND_Z_SPACING = 0.1
GROUND_POS = [0, 0, 0]
GROUND_SIZE = [GROUND_ACTUAL_LENGTH, GROUND_ACTUAL_LENGTH, GROUND_Z_SPACING]
GROUND_RGBA = [0.9, 0.9, 0.9, 1]

# **Create Model**

This section uses the defined parameters to create the model.

In [147]:
# make the model specification:
spec = mj.MjSpec()
spec.modelname = MODEL_NAME

# set compiler settings:
spec.compiler.degree = COMPILER_ANGLE

# set options:
spec.option.timestep = TIMESTEP
spec.option.gravity = GRAVITY
spec.option.integrator = INTEGRATOR

# set visualization options:
spec.visual.scale.framelength = 2.0
spec.visual.scale.framewidth = 0.1
spec.visual.scale.jointlength = 2.0
spec.visual.scale.jointwidth = 0.1

# set default settings:
spec.default.joint.damping = JOINT_DAMPING

# set the visual settings:
spec.visual.quality.shadowsize = SHADOWSIZE
spec.visual.map.znear = ZNEAR
spec.visual.map.zfar = ZFAR

# add the skybox:
spec.add_texture(name = SKYBOX_NAME, 
                 type = SKYBOX_TYPE, 
                 builtin = SKYBOX_BUILTIN, 
                 width = SKYBOX_WIDTH, 
                 height = SKYBOX_HEIGHT, 
                 rgb1 = SKYBOX_RGB1,
                 rgb2 = SKYBOX_RGB2)

# set the worldbody:
spec.worldbody.add_light(name = LIGHT_NAME, 
                         pos = LIGHT_POS, 
                         diffuse = LIGHT_DIFFUSE, 
                         specular = LIGHT_SPECULAR, 
                         ambient = LIGHT_AMBIENT)

spec.worldbody.add_camera(name = CAMERA_NAME, 
                          pos = CAMERA_POS, 
                          )


spec.worldbody.add_geom(name = GROUND_NAME, 
                        type = GROUND_TYPE, 
                        contype = GROUND_CONTYPE, 
                        conaffinity = GROUND_CONAFFINITY,
                        pos = GROUND_POS, 
                        size = GROUND_SIZE, 
                        rgba = GROUND_RGBA)

# add walls:
for name, axis in WALL_INDICES:
    # if its an x-wall:
    if abs(axis[0]):
        WALL_SIZE = [WALL_THICKNESS, GROUND_ACTUAL_LENGTH, WALL_HEIGHT]
        WALL_POS = [axis[0] * (GROUND_ACTUAL_LENGTH - WALL_THICKNESS), 0, WALL_HEIGHT]
    # else its a y-wall:
    else:
        WALL_SIZE = [GROUND_ACTUAL_LENGTH - 2 * WALL_THICKNESS, WALL_THICKNESS, WALL_HEIGHT]
        WALL_POS = [0, axis[1] * (GROUND_ACTUAL_LENGTH - WALL_THICKNESS), WALL_HEIGHT]
    
    spec.worldbody.add_geom(name = name,
                            type = WALL_TYPE,
                            contype = WALL_CONTYPE,
                            conaffinity = WALL_CONAFFINITY,
                            pos = WALL_POS,
                            size = WALL_SIZE)
    
# add the agent:
agent_base = spec.worldbody.add_body(name = "agent_base", pos = [0, 0, 0.01])
agent_base.add_joint(name = "agent_x_slide", type = mj.mjtJoint.mjJNT_SLIDE, axis = [1, 0, 0])
agent_base.add_joint(name = "agent_y_slide", type = mj.mjtJoint.mjJNT_SLIDE, axis = [0, 1, 0])
agent_base.add_geom(name = "agent_base", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 0, conaffinity = 0, rgba = [0, 0, 0, 0])

agent_yaw = agent_base.add_body(name = "agent_yaw", pos = [0, 0, 0])
agent_yaw.add_joint(name = "agent_z_yaw", type = mj.mjtJoint.mjJNT_HINGE, axis = [0, 0, 1])
agent_yaw.add_geom(name = "agent_body", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 1, conaffinity = 1, mass = 0.5, rgba = [1, 0, 0, 1])

# add the actuators:
spec.add_actuator(name = "x_translate", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_x_slide", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])
spec.add_actuator(name = "y_translate", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_y_slide", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])
spec.add_actuator(name = "z_rotation", trntype = mj.mjtTrn.mjTRN_JOINT, target = "agent_z_yaw", ctrlrange = [-0.25, 0.25], gear = [0.05, 0, 0, 0, 0, 0])

# add the task:
task = spec.worldbody.add_body(name = "goal", pos = [0.5, 0.5, 0.01])
task.add_geom(name = "goal", type = mj.mjtGeom.mjGEOM_CYLINDER, size = [0.1, 0.01, 0], contype = 0, conaffinity = 0, rgba = [0, 1, 0, 1])

# compile into model:
model = spec.compile()

load a real model so that I can check what each value means (example "integrator = 2" is that RKF or something else)

In [148]:
# model = mj.MjModel.from_xml_path("assets/env_test_matt.xml")
# print("loaded xml file")

data = mujoco.MjData(model)

# launch a passive window using the model and the data contained within:
with mujoco.viewer.launch_passive(model, data) as viewer:
    # switch the camera:
    viewer.cam.type = mj.mjtCamera.mjCAMERA_FIXED
    viewer.cam.fixedcamid = model.camera(CAMERA_NAME).id

    # enable viewer options:
    viewer.opt.frame = mj.mjtFrame.mjFRAME_BODY
    viewer.opt.flags[mj.mjtVisFlag.mjVIS_JOINT] = True
    
    # while viewer is active, step the model every timestep:
    while viewer.is_running():
        mujoco.mj_step(model, data)
        viewer.sync()